# c05-???

*Purpose*: (Apply control charts, Pr modeling techniques)


In [ ]:
import grama as gr
import numpy as np
import pandas as pd
import time
DF = gr.Intention()
%matplotlib inline

filename_data = "./data/c05-data.csv"

# Stang


In [ ]:
from grama.data import df_stang
df_stang.head()

In [ ]:
(
    df_stang
    >> gr.pt_xbs(group="thick", var="E")
)

- The variability is under control
- The mean is not under control; in particular the thickest plates have a much lower elasticity

# True Model Setup


In [ ]:
from grama.models import make_plate_buckle
md_plate = make_plate_buckle()
md_plate

## Data generation

Want a dataset to have students practice using control charts; deliberately introduce some out-of-control observations in the dataset.


In [ ]:
np.random.seed(101)

n_batch = 10
mg_standard = gr.marg_mom("norm", mean=0, sd=1)

# Base properties; mean and sd differences across operators and plate thicknesses
df_base = (
    gr.df_make(
        t=[1/4, 1/8],
        E_base=[1.0e4, 1.2e4],
        COV=[0.03, 0.03],
    )
    >> gr.tf_outer(gr.df_make(
        machine=["A", "B", "C", "D", "E", "F"],
        eff=[0, 0, 0.06, 0, 0, 0],
    ))
    >> gr.tf_mutate(E_mean=DF.E_base * (1 + DF.eff))
)

df_noise = gr.df_make(
    id_measurement=["g", "h", "i", "j", "k", "l"],
    noise_sd=[4e2, 8e2, 4e2, 4e2, 4e2, 4e2],
)

df_data = gr.df_grid()
for i in range(df_base.shape[0]):
    # Initialize model for true material properties
    md_tmp = (
        gr.Model()
        >> gr.cp_marginals(
            E = gr.marg_mom("lognorm", floc=0, mean=df_base.E_mean[i], cov=df_base.COV[i]),
            mu = gr.marg_mom("lognorm", floc=0, mean=0.32, cov=0.03),
        )
        >> gr.cp_copula_independence()
    )

    # "True" material properties
    df_true = (
        md_tmp
        >> gr.ev_sample(n=n_batch, df_det="nom", skip=True)
        >> gr.tf_mutate(t=df_base.t[i], id_machine=df_base.machine[i])
    )

    # Record values
    df_data = (
        df_data
        >> gr.tf_bind_rows(df_true)
    )


# Measured properties
n_total = df_data.shape[0] * df_noise.shape[0]
df_data = (
    df_data
    >> gr.tf_mutate(id_specimen=DF.index)
    >> gr.tf_outer(df_noise)
    >> gr.tf_mutate(z=mg_standard.r(n_total), z2=mg_standard.r(n_total))
    >> gr.tf_mutate(
        E=DF.E + DF.noise_sd * DF.z,
        mu=DF.mu + 0.005 * DF.z2,
    )
    >> gr.tf_drop("z", "z2", "noise_sd")
)

## Write data to disk
df_data.to_csv(filename_data, index=False)

## Printout
df_data

In [ ]:
(
    df_data
    >> gr.tf_arrange(DF.id_specimen)
    >> gr.tf_mutate(id=DF.index)
    >> gr.ggplot(gr.aes("id", "E"))
    + gr.geom_point()
)

In [ ]:
(
    df_data
    >> gr.tf_arrange(DF.id_specimen)
    >> gr.tf_mutate(id=DF.index)
    >> gr.ggplot(gr.aes("id", "mu"))
    + gr.geom_point()
)

In [ ]:
(
    df_data
    >> gr.tf_group_by(DF.t)
    >> gr.tf_summarize(
        E_mean=gr.mean(DF.E),
        E_sd=gr.sd(DF.E),
    )
    >> gr.tf_mutate(E_cov=DF.E_sd/DF.E_mean)
)

# Data Analysis


## Real vs Error


In [ ]:
df_real = (
    df_data
    >> gr.tf_group_by(DF.t, DF.id_specimen, DF.id_machine)
    >> gr.tf_summarize(
        E=gr.mean(DF.E),
        mu=gr.mean(DF.mu),
    )
    >> gr.tf_ungroup()
    >> gr.tf_arrange(DF.id_machine)
)
df_real

## Xbar and S charts

### 1/4 Plates


In [ ]:
(
    df_real
    >> gr.tf_filter(DF.t == 1/4)
    >> gr.pt_xbs(group="id_machine", var="E")
)

In [ ]:
(
    df_data
    >> gr.tf_filter(DF.t == 1/4)
    >> gr.pt_xbs(group="id_measurement", var="E")
)

### 1/8 Plates


In [ ]:
(
    df_real
    >> gr.tf_filter(DF.t == 1/8)
    >> gr.pt_xbs(group="id_machine", var="E")
)

In [ ]:
(
    df_data
    >> gr.tf_filter(DF.t == 1/8)
    >> gr.pt_xbs(group="id_measurement", var="E")
)